<a href="https://colab.research.google.com/github/IsraelAbebe/Personal-Projects-and-Exercises/blob/master/Tutorials/Pytorch_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


What is PyTorch?
================

It’s a Python-based scientific computing package targeted at two sets of
audiences:

-  A replacement for NumPy to use the power of GPUs
-  a deep learning research platform that provides maximum flexibility
   and speed

1.Getting Started
---------------

Tensors
^^^^^^^

Tensors are similar to NumPy’s ndarrays, with the addition being that
Tensors can also be used on a GPU to accelerate computing.



In [0]:
import torch
import matplotlib.pyplot as plt
import numpy as np

In [138]:
# torch.empty()
torch.empty(2,3)

tensor([[-2.4318e-17,  0.0000e+00,  4.4842e-44],
        [ 0.0000e+00,         nan,  1.6115e-43]])

In [139]:
# torch.rand() 
torch.rand(3,3)

tensor([[0.3620, 0.3549, 0.8494],
        [0.6823, 0.9043, 0.8429],
        [0.1639, 0.0811, 0.6133]])

In [0]:
# torch.zeros()

In [0]:
# torch.tensor()

In [0]:
# torch.randn_like()    # override dtype!

In [0]:
# size()

In [0]:
# torch.add()

In [0]:
# item()

In [0]:
# .item()

In [0]:
# a.numpy()
# torch.from_numpy()

In [0]:
# torch.cuda.is_available()

In [0]:
# if torch.cuda.is_available():
#     device = torch.device("cuda")          
#     y = torch.ones_like(x, device=device)  
#     x = x.to(device)   
#     z = x + y
#     print(z)
#     print(z.to("cpu", torch.double)) 

# 2.Autograd: Automatic Differentiation


``` code
            def gradient(y,y_pred,x):
                    return np.dot(x.T,(y_pred-y))

```

In [164]:
x = torch.rand(1, requires_grad=True)
y = torch.rand(1, requires_grad=True)
z= torch.rand(1, requires_grad=True)

print(x.grad,y.grad,z.grad)
print(x,y)

None None None
tensor([0.4492], requires_grad=True) tensor([0.7218], requires_grad=True)


In [165]:
one = x * y
two = one + z
three = torch.sigmoid(two)

print(three)

tensor([0.6071], grad_fn=<SigmoidBackward>)


In [166]:
three.backward()

print(x.grad,y.grad,z.grad)

tensor([0.1722]) tensor([0.1071]) tensor([0.2385])


# NEURAL NETWORKS

In [0]:
def get_shape(h,p=0,k=3,s=1):
    return ((h+(2*p)-k)/s)+1

In [168]:
get_shape(110,k=5)/2

53.0

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [0]:
class Classification(nn.Module):
    def __init__(self):
        super(Classification,self).__init__()
        self.conv1 = nn.Conv2d(in_channels=3,out_channels=64,kernel_size=5)
        self.conv2 = nn.Conv2d(in_channels=64,out_channels=64,kernel_size=5)
        
        self.fc1 = nn.Linear(in_features=64*53*53,out_features=120)
        self.fc2 = nn.Linear(in_features = 120,out_features = 64)
        self.fc3 = nn.Linear(in_features = 64,out_features = 10)


    def forward(self,x):
        x = F.max_pool2d(F.relu(self.conv1(x)),2)
        x = F.max_pool2d(F.relu(self.conv2(x)),2)

        print(self.num_flat_features(x))
        x = x.view(-1,self.num_flat_features(x))

        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)

        return x

    def num_flat_features(self,x):
        size = x.size()[1:]
        num_features = 1
        for s in size:
            num_features *= s
        
        return num_features



In [171]:
net =  Classification()
print(net)

Classification(
  (conv1): Conv2d(3, 64, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(64, 64, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=179776, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=64, bias=True)
  (fc3): Linear(in_features=64, out_features=10, bias=True)
)


In [0]:
# from torchsummary import summary
# summary(net.to('cpu'), (3, 224, 224))

In [173]:
param = list(net.parameters())
param[0][0][0].size()

torch.Size([5, 5])

In [0]:
# filter_0 = param[0][0][0].detach().numpy()
# plt.imshow(filter_0)

In [194]:
input = torch.randn(1, 3, 224, 224)
out = net(input)
print(out)

179776
tensor([[-0.1256, -0.0637, -0.0915, -0.0405, -0.0882,  0.0959, -0.0413,  0.1461,
          0.0041, -0.0907]], grad_fn=<AddmmBackward>)


In [0]:
# Zero the gradient buffers of all parameters and backprops with random gradients:

net.zero_grad()
out.backward(torch.randn(1, 10))

In [196]:
# Loss Function
output = net(input)
target = torch.randn(10)  
target = target.view(1, -1) 

criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

179776
tensor(1.8025, grad_fn=<MseLossBackward>)


In [197]:
loss.backward()
net.conv1.weight.grad

torch.Size([64, 3, 5, 5])

In [199]:
import torch.optim as optim

input = torch.randn(1, 3, 224, 224)
out = net(input)
print(out)

optimizer = optim.SGD(net.parameters(), lr=0.01)

optimizer.zero_grad() 
print(net.conv1.bias)  
output = net(input)
loss = criterion(output, target)
loss.backward()
optimizer.step() 
print(net.conv1.bias) 

179776
tensor([[-2.0923,  0.3265,  0.4433,  2.2555, -0.2949,  1.1524,  0.6555,  1.2390,
         -0.4443, -1.6031]], grad_fn=<AddmmBackward>)
Parameter containing:
tensor([-0.0328, -0.0512,  0.0645, -0.0345,  0.0683, -0.0707,  0.0279,  0.0937,
        -0.0859, -0.0148, -0.0537,  0.0734,  0.0046, -0.0675, -0.0720, -0.0006,
        -0.1146, -0.1059,  0.1154, -0.0699,  0.1061, -0.0968, -0.0180, -0.0569,
         0.0477,  0.1135, -0.0295, -0.1017,  0.0095, -0.0252, -0.0478,  0.0445,
        -0.0269,  0.0277,  0.0122, -0.0981,  0.0366, -0.0124,  0.0469,  0.0044,
        -0.0669, -0.0703, -0.1102, -0.0610, -0.0520, -0.1039,  0.1039,  0.0244,
         0.0852,  0.0355,  0.0468, -0.0434, -0.0129,  0.0222, -0.0307,  0.0999,
         0.0409,  0.0089, -0.0709,  0.0717, -0.0214,  0.0039,  0.0483, -0.0832],
       requires_grad=True)
179776
Parameter containing:
tensor([-0.0329, -0.0513,  0.0643, -0.0345,  0.0680, -0.0707,  0.0279,  0.0939,
        -0.0860, -0.0151, -0.0535,  0.0733,  0.0046, -0.067

# TRAINING A CLASSIFIER 

[Cifar Data](https://www.cs.toronto.edu/~kriz/cifar.html)

In [200]:
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html

SyntaxError: ignored